### Konstantina Andronikou 

## This Notebook implements Predicate Ellipsis 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
#removing warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John saw three dogs and Mary two.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'saw',
   'description': '[ARG0: John] [V: saw] [ARG1: three dogs and Mary two] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']}],
 'words': ['John', 'saw', 'three', 'dogs', 'and', 'Mary', 'two', '.']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["John saw three dogs and Mary two"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'saw',
     'description': '[ARG0: John] [V: saw] [ARG1: three dogs and Mary two]',
     'tags': ['B-ARG0',
      'B-V',
      'B-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1']}],
   'words': ['John', 'saw', 'three', 'dogs', 'and', 'Mary', 'two']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name1']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} saw three dogs and {first_name1} two.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name1': 'Louise', 'first_name': 'Sara'}, {'first_name1': 'Alexandra', 'first_name': 'William'}, {'first_name1': 'Karen', 'first_name': 'Jane'}, {'first_name1': 'Steven', 'first_name': 'Sophie'}, {'first_name1': 'Ron', 'first_name': 'Jack'}, {'first_name1': 'Lucy', 'first_name': 'Ray'}, {'first_name1': 'Carol', 'first_name': 'Ryan'}, {'first_name1': 'Tim', 'first_name': 'Eleanor'}, {'first_name1': 'Barbara', 'first_name': 'Sarah'}, {'first_name1': 'Louise', 'first_name': 'Andrew'}, {'first_name1': 'Alexandra', 'first_name': 'Nick'}, {'first_name1': 'Eleanor', 'first_name': 'Adam'}, {'first_name1': 'Evelyn', 'first_name': 'Barbara'}, {'first_name1': 'Peter', 'first_name': 'Edith'}, {'first_name1': 'Nick', 'first_name': 'Francis'}, {'first_name1': 'Fiona', 'first_name': 'Leslie'}, {'first_name1': 'Scott', 'first_name': 'Annie'}, {'first_name1': 'Dorothy', 'first_name': 'Jennifer'}, {'first_name1': 'Alison', 'first_name': 'Robin'}, {'first_name1': 'Matt', 'first_name': 'Flore

In [12]:
with open('Dataset_BERT/Predicate_Ellipsis.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Predicate_Ellipsis_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 996 examples
Test cases:      996
Fails (rate):    996 (100.0%)

Example fails:
[ARG0: Angela] [V: saw] [ARG1: three dogs and Johnny two] .
----
[ARG0: Catherine] [V: saw] [ARG1: three dogs and Robert two] .
----
[ARG0: Alex] [V: saw] [ARG1: three dogs] and Sophie two .
----
['Sara', 'saw', 'three', 'dogs', 'and', 'Louise', 'two', '.'] [False]
['William', 'saw', 'three', 'dogs', 'and', 'Alexandra', 'two', '.'] [False]
['Jane', 'saw', 'three', 'dogs', 'and', 'Karen', 'two', '.'] [False]
['Sophie', 'saw', 'three', 'dogs', 'and', 'Steven', 'two', '.'] [False]
['Jack', 'saw', 'three', 'dogs', 'and', 'Ron', 'two', '.'] [False]
['Ray', 'saw', 'three', 'dogs', 'and', 'Lucy', 'two', '.'] [False]
['Ryan', 'saw', 'three', 'dogs', 'and', 'Carol', 'two', '.'] [False]
['Eleanor', 'saw', 'three', 'dogs', 'and', 'Tim', 'two', '.'] [False]
['Sarah', 'saw', 'three', 'dogs', 'and', 'Barbara', 'two', '.'] [False]
['Andrew', 'saw', 'three', 'dogs', 'and', 'Louise', 'two', '.'] [False]
['Nick', 

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Predicate_Ellipsis_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 